# Lecture Plan
 
1. Langchain Agents
2. Langchain Tools
   - built in
   - custom
   - tool kits
3. Memroy
   - Chat Message History
   - Conversational Buffer Memory
   - Conversational Window Memory
   - Conversational Summary Memory

#### Agents

Agents are used to take dynamic actions using a language model.

The agent requires:

1. A base llm
2. tool/tools
3. system message (optional)
4. memory

#### Tools

- builtin tools
- tool kits
- custom tools


reference: [langchin handbook](https://www.pinecone.io/learn/series/langchain/)

In [ ]:
! pip install duckduckgo-search langchain_openai

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.llms import OpenAI
from langchain.llms.deepinfra import DeepInfra
from langchain_experimental.chat_models import Llama2Chat

llm = DeepInfra(
        model_id="meta-llama/Llama-2-70b-chat-hf"

)

model = Llama2Chat(llm=llm)



# llm = OpenAI(temperature=0.0)
# model = ChatOpenAI(model="gpt-3.5-turbo" ,temperature=0.0)

In [7]:
from langchain.agents import initialize_agent, load_tools, AgentType, AgentExecutor, create_react_agent


tools = load_tools(
    ['llm-math', 'ddg-search'],
    llm = llm

)


In [16]:
from langchain.memory import ConversationBufferMemory
agent = initialize_agent(
    agent= AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    verbose=True,
    llm=model,
    handle_parsing_errors = True,
    max_iterations = 3,
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

    
)

agent.bind(stop="Observation")

RunnableBinding(bound=AgentExecutor(memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history'), verbose=True, tags=['conversational-react-description'], agent=ConversationalAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informat

In [17]:
agent.invoke({"what is log(67)"})



> Entering new AgentExecutor chain...
  Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: log(67)
Observation: The result of log(67) is 2.2364785551641533

Thought: Do I need to use a tool? No
AI: The answer to your question is 2.2364785551641533.

> Finished chain.


{'input': {'what is log(67)'},
 'chat_history': [HumanMessage(content=['what is log(67)']),
  AIMessage(content='The answer to your question is 2.2364785551641533.')],
 'output': 'The answer to your question is 2.2364785551641533.'}

In [ ]:
! pip install redis

In [13]:
from langchain.agents import ConversationalAgent
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.memory.chat_message_histories import UpstashRedisChatMessageHistory, SQLChatMessageHistory
import os
from datetime import datetime

# chat_history = UpstashRedisChatMessageHistory(url = os.getenv('UPSTASH_REDIS_REST_URL'), session_id  = 'trng-1855')
# chat_history = SQLChatMessageHistory(session_id=datetime.now().strftime("%m/%d/%Y, %H:%M"), connection_string=os.getenv('CHAT_SQL_CONNECTION_URL'))
# memory = ConversationBufferMemory(memory_key='chat_history', chat_memory= chat_history, return_messages=True)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


system_message = """\
<system>You are an honest and helpfull sales agent for an internet provider company called brighspeed</system>

<user>Question</user>

Answer:
"""

agent = ConversationalAgent.from_llm_and_tools(

    tools=tools,
    llm= model,
    system_mesage = system_message
    
)

agent_executor = AgentExecutor.from_agent_and_tools(

    agent=agent,
    tools=tools,
    memory=memory,
    handle_parsing_errors = True,
    verbose = True,
    max_iterations=3

    
)

memory.clear()

In [16]:
agent_executor.invoke({"What is Brightspeed?"})



> Entering new AgentExecutor chain...
  Thought: Do I need to use a tool? Yes
Action: duckduckgo_search
Action Input: 'Brightspeed'
Observation: According to DuckDuckGo Search, Brightspeed is a new fiber-optic internet service provider that is currently being rolled out in select areas across the United States. It is unclear what specific areas are currently being serviced by Brightspeed, but it appears to be a relatively new and rapidly expanding company.

Thought: Do I need to use a tool? No
AI: Brightspeed appears to be a new fiber-optic internet service provider that is currently being rolled out in select areas across the United States. It is unclear what specific areas are currently being serviced by Brightspeed, but it appears to be a relatively new and rapidly expanding company. Can I help you with anything else?

> Finished chain.


{'input': {'What is Brightspeed?'},
 'chat_history': [HumanMessage(content=['What is Brightspeed?']),
  AIMessage(content='Brightspeed appears to be a new fiber-optic internet service provider that is currently being rolled out in select areas across the United States. It is unclear what specific areas are currently being serviced by Brightspeed, but it appears to be a relatively new and rapidly expanding company. Can I help you with anything else?')],
 'output': 'Brightspeed appears to be a new fiber-optic internet service provider that is currently being rolled out in select areas across the United States. It is unclear what specific areas are currently being serviced by Brightspeed, but it appears to be a relatively new and rapidly expanding company. Can I help you with anything else?'}

In [18]:
from pinecone import Pinecone

key = os.getenv('PINECONE_API_KEY')
client = Pinecone(api_key=key)

index = client.Index('trng-index')

c:\TrainingMaterial\generative-ai\genai-material\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [20]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name='thenlper/gte-large'
)


In [21]:
from langchain.agents import Tool
from langchain.vectorstores.pinecone import Pinecone
from langchain.prompts import PromptTemplate

vectordb = Pinecone(index, embedding=embeddings.embed_query, text_key="text")

prompt = PromptTemplate.from_template(template="query the vector db for results")

chain = prompt | llm
tool = Tool(
    name="vector db tool",
    description = "used to retreive information related to a brightspeed",
    func = vectordb.similarity_search,
    retreiver_top_k=3
)

tools.append(tool)

c:\TrainingMaterial\generative-ai\genai-material\.venv\Lib\site-packages\langchain_community\vectorstores\pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(
